## Using Memory: Personalized Context Through Persistent Memory

In [13]:
!pip install --upgrade strands-agents==0.1.7 strands-agents-tools==0.1.5 mem0ai duckduckgo-search faiss-cpu
!pip install --upgrade boto3

/Users/giryoong/Projects/github/generative-ai-on-amazon-sagemaker-june13/.conda/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=80458) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Looking in indexes: https://pypi.org/simple, https://plugin.us-east-1.prod.workshops.aws
Looking in indexes: https://pypi.org/simple, https://plugin.us-east-1.prod.workshops.aws


In [14]:
# Import Required Libraries
import os
from strands import Agent, tool
from strands_tools import mem0_memory
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import DuckDuckGoSearchException, RatelimitException

os.environ["AWS_DEFAULT_REGION"]="us-west-2"

In [15]:
# Define a focused system prompt for memory operations
SYSTEM_PROMPT = """You are a helpful personal assistant for a user. Your task is to assist the user by providing personalized responses based on their history. 

Capabilities:
- You can store information using the mem0_memory tool (action="store").
- You can retrieve relevant memories using the mem0_memory tool (action="retrieve").
- You can use duckduckgo_search to find information on the web.

Key Rules:
- Be conversational and natural in your responses.
- Always retrieve memories before responding to the user and use them to inform your response.
- Store any new user information and user preferences in mem0_memory.
- Only share relevant information.
- Politely indicate when you don't have the information.
"""

In [16]:
@tool
def websearch(
    keywords: str,
    region: str = "us-en",
    max_results: int | None = None,
) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DuckDuckGoSearchException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

In [17]:
# Create an agent with memory, websearch tool
USER_ID = "new_user" # Replace with actual user ID

memory_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[mem0_memory, websearch],
)

In [26]:
# Store initial memories to demonstrate retrieval
memory_agent.tool.mem0_memory(
    action="store", content=f"The user's name is {USER_ID}.", user_id=USER_ID
)
memory_agent.tool.mem0_memory(
    action="store", 
    content="I like to drink tea more than coffee.", 
    user_id=USER_ID
)

╭─────────────────────────────────────────── Memory for user new_user ────────────────────────────────────────────╮
│ The user's name is new_user.                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── ❌ Memory Operation Error ───────────────────────────────────────────╮
│ An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token      │
│ included in the request is invalid.                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Memory for user new_user ────────────────────────────────────────────╮
│ I like to drink tea more than coffee.                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── ❌ Memory Operation Error ───────────────────────────────────────────╮
│ An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token      │
│ included in the request is invalid.                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'toolUseId': 'tooluse_mem0_memory_294523918',
 'status': 'error',
 'content': [{'text': 'Error: An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token included in the request is invalid.'}]}

In [19]:
# Retrieve memories
retrieved_memories = memory_agent.tool.mem0_memory(
    action="retrieve", query="What is the user's name?", user_id=USER_ID
)
print("Retrieved Memories:", retrieved_memories)

╭─────────────────────────────────────────── ❌ Memory Operation Error ───────────────────────────────────────────╮
│ Error getting embedding from AWS Bedrock: An error occurred (UnrecognizedClientException) when calling the      │
│ InvokeModel operation: The security token included in the request is invalid.                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Retrieved Memories: {'toolUseId': 'tooluse_mem0_memory_964572442', 'status': 'error', 'content': [{'text': 'Error: Error getting embedding from AWS Bedrock: An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token included in the request is invalid.'}]}


In [20]:
# Retrieve memories about preferences
memory_agent.tool.mem0_memory(
    action="retrieve",
    query="What are the my drink preferences?",
    user_id=USER_ID
)

╭─────────────────────────────────────────── ❌ Memory Operation Error ───────────────────────────────────────────╮
│ Error getting embedding from AWS Bedrock: An error occurred (UnrecognizedClientException) when calling the      │
│ InvokeModel operation: The security token included in the request is invalid.                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'toolUseId': 'tooluse_mem0_memory_933100576',
 'status': 'error',
 'content': [{'text': 'Error: Error getting embedding from AWS Bedrock: An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token included in the request is invalid.'}]}

In [21]:
# Ask the agent a question
response = memory_agent("What are the events happening in the New York today?")
print(response)

You are a helpful personal assistant for a user. Your task is to assist the user by providing personalized responses based on their history. 

Capabilities:
- You can store information using the mem0_memory tool (action="store").
- You can retrieve relevant memories using the mem0_memory tool (action="retrieve").
- You can use duckduckgo_search to find information on the web.

Key Rules:
- Be conversational and natural in your responses.
- Always retrieve memories before responding to the user and use them to inform your response.
- Store any new user information and user preferences in mem0_memory.
- Only share relevant information.
- Politely indicate when you don't have the information.
 [{'role': 'user', 'content': [{'text': 'agent.tool.mem0_memory direct tool call.\nInput parameters: {"action": "store", "content": "The user\'s name is new_user.", "user_id": "new_user"}\n'}]}, {'role': 'assistant', 'content': [{'toolUse': {'toolUseId': 'tooluse_mem0_memory_704221315', 'name': 'mem0

EventStreamError: An error occurred (validationException) when calling the ConverseStream operation: The model returned the following errors: tools.0.custom.input_schema: input_schema does not support oneOf, allOf, or anyOf at the top level

In [ ]:
# List all stored memories
print("All Stored Memories:")
all_memories = memory_agent.tool.mem0_memory(
    action="list", user_id=USER_ID
)

All Stored Memories:


╭───────────────────────────────────────────────── Memories List ─────────────────────────────────────────────────╮
│                                                    Memories                                                     │
│ ┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃ ID             ┃ Memory                                             ┃ Created At      ┃ User ID  ┃ Metadata ┃ │
│ ┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ 94e68f3a-0f2a… │ Name is new_user                                   │ 2025-06-16T21:… │ new_user │ None     │ │
│ │ 17bf9c7e-14c4… │ Prefers tea over coffee                            │ 2025-06-16T21:… │ new_user │ None     │ │
│ └────────────────┴────────────────────────────────────────────────────┴─────────────────┴──────────┴──────────┘ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯